In [3]:
from fastai.vision.all import *
from pathlib import Path
import numpy as np
from loss_functions.triplet_loss import TripletLoss
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
class L2_norm(nn.Module):
    def __init__(self):
        super(L2_norm, self).__init__()

    def forward(self, x):
        return F.normalize(x, p=2, dim=-1)


data loader

In [5]:
path = "/mnt/sda/nikhil/pbb/iml/"
train_path = "/mnt/sda/nikhil/pbb/iml/train/"
valid_path = "/mnt/sda/nikhil/pbb/iml/valid/"


# Define the transformations and batch size
item_tfms = Resize(224)
batch_size = 64

# Define the data block
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_items=get_image_files,
                   get_y=parent_label,
                   splitter=GrandparentSplitter(train_name='train', valid_name='valid'))

# Create the dataloaders for training and validation sets
dls = dblock.dataloaders(path, item_tfms=item_tfms, bs=batch_size)
print(dls.train_ds.items[0])
print(dls.valid_ds.items[0])

/mnt/sda/nikhil/pbb/iml/train/mid/04995-base_zl_20_03283_tx_268729_ty_595319__from_1S_134-135_-pine-middle.jpg
/mnt/sda/nikhil/pbb/iml/valid/mid/23772-base_zl_20_tx_893080_ty_409455__from_3s_119_136-pine-middle.jpg


In [6]:
learn = vision_learner(dls, resnet50, pretrained=True)

# Modify the second layer of the ResNet-50 model
layers = learn.model[1]
learn.model[1] = nn.Sequential(layers[0], layers[1], layers[2], layers[3], nn.Linear(in_features=4096, out_features=128, bias=False), L2_norm()).to(device)

# Set the model directory
learn.model_dir = '/mnt/sda/nikhil/pbb/imlmodels/'

/home/nikhil/anaconda3/envs/ptc/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [7]:
learn.loss_func = TripletLoss(device)
learn.load('stage1_weights')

load data to get embeddings

In [8]:
import glob
from tqdm import tqdm
path = "/mnt/sda/nikhil/pbb/iml/"

def get_embeddings_and_labels(path):
    images = glob.glob(path + '*/*')
    embeddings = []
    labels = []
    for image in tqdm(images, desc="Processing images"):
        img = load_image(image)
        pred = learn.predict(img)
        embedding = pred[-1].detach().numpy()
        embeddings.append(embedding)
        labels.append(image.split('/')[-2])  # Assumes folder name is the label
    return embeddings, labels



train_path = "/mnt/sda/nikhil/pbb/iml/train/"
valid_path = "/mnt/sda/nikhil/pbb/iml/valid/"
test_path = "/mnt/sda/nikhil/pbb/iml/test/"

x_train, y_train = get_embeddings_and_labels(train_path)
x_train, y_train = np.asarray(x_train), np.asarray(y_train)
np.save('imlstagetrain_embeddings.npy', x_train)
np.save('imlstagetrain_labels.npy', y_train)

x_valid, y_valid = get_embeddings_and_labels(valid_path)
x_valid, y_valid = np.asarray(x_valid), np.asarray(y_valid)
np.save('imlstagevalid_embeddings.npy', x_valid)
np.save('imlstagevalid_labels.npy', y_valid)

x_test, y_test = get_embeddings_and_labels(test_path)
x_test, y_test = np.asarray(x_test), np.asarray(y_test)
np.save('imlstagetest_embeddings.npy', x_test)
np.save('imlstagetest_labels.npy', y_test)
print("Done")


Processing images:  86%|████████▌ | 2555/2976 [11:17<02:07,  3.30it/s]

Processing images:  86%|████████▌ | 2556/2976 [11:17<02:04,  3.37it/s]

Processing images:  86%|████████▌ | 2557/2976 [11:17<02:11,  3.18it/s]

Processing images:  86%|████████▌ | 2558/2976 [11:18<02:01,  3.45it/s]

Processing images:  86%|████████▌ | 2559/2976 [11:18<01:57,  3.56it/s]

...

Processing images: 100%|█████████▉| 2973/2976 [13:02<00:00,  3.72it/s]

Processing images: 100%|█████████▉| 2974/2976 [13:03<00:00,  3.71it/s]

Processing images: 100%|█████████▉| 2975/2976 [13:03<00:00,  3.97it/s]

Processing images: 100%|██████████| 2976/2976 [13:03<00:00,  3.80it/s]

Done


gpt says this code will be faster than the one above.

In [ ]:
# import glob
# from tqdm import tqdm
# import torch
# import numpy as np
# from fastai.vision.all import *
# from pathlib import Path

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# path = "/mnt/sda/nikhil/pbb/iml/"

# def get_embeddings_and_labels(path):
#     images = glob.glob(path + '*/*')
#     embeddings = []
#     labels = []
#     for image in tqdm(images, desc="Processing images"):
#         img = load_image(image).to(device)  # Move image to GPU
#         pred = learn.predict(img)
#         embedding = pred[-1].detach().cpu().numpy()  # Move embedding to CPU
#         embeddings.append(embedding)
#         labels.append(image.split('/')[-2])  # Assumes folder name is the label
#     return embeddings, labels

# # Load the Fastai learner model
# learn = load_learner(path)

# train_path = "/mnt/sda/nikhil/pbb/iml/train/"
# valid_path = "/mnt/sda/nikhil/pbb/iml/valid/"
# test_path = "/mnt/sda/nikhil/pbb/iml/test/"

# x_train, y_train = get_embeddings_and_labels(train_path)
# x_train, y_train = np.asarray(x_train), np.asarray(y_train)
# np.save('imlstagetrain_embeddings.npy', x_train)
# np.save('imlstagetrain_labels.npy', y_train)

# x_valid, y_valid = get_embeddings_and_labels(valid_path)
# x_valid, y_valid = np.asarray(x_valid), np.asarray(y_valid)
# np.save('imlstagevalid_embeddings.npy', x_valid)
# np.save('imlstagevalid_labels.npy', y_valid)

# x_test, y_test = get_embeddings_and_labels(test_path)
# x_test, y_test = np.asarray(x_test), np.asarray(y_test)
# np.save('imlstagetest_embeddings.npy', x_test)
# np.save('imlstagetest_labels.npy', y_test)

# print("Done")